In [7]:
!/Users/pengyuanlong/.pyenv/versions/3.10.4/bin/python3.10 -m pip install --upgrade pip
!pip install robotexclusionrulesparser

In [11]:
import sys
print(sys.executable)
import sys
sys.path.append('/Users/pengyuanlong/.pyenv/versions/3.10.4/lib/python3.10/site-packages')
import robotexclusionrulesparser as rerp

/Users/pengyuanlong/opt/anaconda3/bin/python


In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import json
import time
import random
import robotexclusionrulesparser as rerp

def can_fetch(url, user_agent='*'):
    """Vérifier si l'extraction de cette URL est autoriséeL"""
    domain = url.split("/")[2]
    robots_url = f"https://{domain}/robots.txt"
    try:
        rerp_parser = rerp.RobotExclusionRulesParser()
        robots_txt = requests.get(robots_url).text
        rerp_parser.parse(robots_txt)
        return rerp_parser.is_allowed(user_agent, url)
    except requests.RequestException as e:
        print(f"Error fetching robots.txt from {robots_url}: {e}")
        return False

def fetch_data(url):
    """Récupérer le contenu de la page Web à partir de l'URL donnée"""
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.text
    except requests.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return None

def parse_data(html, counts):
    """Analyser les données de la page Web, extraire le titre du livre, l'auteur, la note et le texte de la critique du livre, et gérer les données manquantes éventuelles"""
    soup = BeautifulSoup(html, 'html.parser')
    review_data = []

    # Trouver le titre du livre et l'auteur.
    try:
        book_title = soup.find('h1', itemprop='name').get_text(strip=True)
    except AttributeError:
        book_title = "Unknown"

    try:
        author = soup.find('span', itemprop='name').get_text(strip=True)
    except AttributeError:
        author = "Unknown"

    # Trouver tous les blocs de commentaires.
    reviews = soup.find_all('div', class_='post_con')
    for review in reviews:
        try:
            review_text = review.find('div', class_='cri_corps_critique').get_text(strip=True)
        except AttributeError:
            review_text = "No review available."

        try:
            rating = float(review.find('div', class_='rateit')['data-rateit-value'])
        except (TypeError, ValueError, AttributeError):
            rating = None

        if rating is not None:  # Ajouter à la liste de données uniquement lorsque la note est disponible.
            label = assign_label(rating)
            if counts[label] < 50:  # Ajouter uniquement lorsque le nombre correspondant d'étiquettes est inférieur à 50
                review_data.append([book_title, author, review_text, rating, label])
                counts[label] += 1
                if all(count == 50 for count in counts.values()):
                    break

    return review_data

def assign_label(rating):
    """Définir les étiquettes en fonction des notes"""
    if rating is None:
        return 0  
    elif rating >= 4.0:
        return 1  
    elif rating <= 2.5:
        return -1 
    else:
        return 0 

def save_data(data, filename):
    """Enregistrer les données dans un fichier JSON"""
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def main():
    urls = [
        'https://www.babelio.com/livres/Rousselet-La-belle-histoire-des-maths/121795',  # 示例 URL
        'https://www.babelio.com/livres/Saint-Exupery-Le-Petit-Prince/36712',  # 示例 URL
        'https://www.babelio.com/livres/Hugo-Les-Miserables/1329123',  # 示例 URL
        'https://www.babelio.com/livres/Duras-Lamant/5772',  # 示例 URL
        'https://www.babelio.com/livres/Camus-Letranger/3874',  # 示例 URL
        'https://www.babelio.com/livres/Hugo-Les-Travailleurs-de-la-mer/7119',
        'https://www.babelio.com/livres/Camus-La-Peste/313209',
        'https://www.babelio.com/livres/Camus-La-Chute/3631',
        'https://www.babelio.com/livres/Proust-A-la-recherche-du-temps-perdu-tome-1--Du-cote-de-/822463',
        'https://www.babelio.com/livres/Stendhal-Oeuvres-romanesques-completes-tome-1/99891',
        'https://www.babelio.com/livres/Balzac-La-comedie-humaine-La-Pleiade-tome-1/711990',
        'https://www.babelio.com/livres/Hugo-Quatrevingt-Treize/7115',
        'https://www.babelio.com/livres/Hugo-LHomme-qui-rit/3295',
        'https://www.babelio.com/livres/Stendhal-Le-Rouge-et-le-Noir/2908',
        'https://www.babelio.com/livres/Dumas-Les-Trois-Mousquetaires/1397512',
        'https://www.babelio.com/livres/Balzac-Eugenie-Grandet/78277',
        'https://www.babelio.com/livres/Maupassant-Mademoiselle-Fifi-et-autres-nouvelles/731832',
        'https://www.babelio.com/livres/Verne-Le-tour-du-monde-en-80-jours/1361536',
        'https://www.babelio.com/livres/Verne-LIle-mysterieuse/9547',
        'https://www.babelio.com/livres/Verne-Vingt-mille-lieues-sous-les-mers/5760',
    ]

    counts = {1: 0, 0: 0, -1: 0} 
    all_reviews = []

    for url in urls:
        page = 1
        while all(count < 50 for count in counts.values()):
            critique_url = f"{url}/critiques?a=a&pageN={page}"
            if can_fetch(critique_url):
                html = fetch_data(critique_url)
                if html:
                    data = parse_data(html, counts)
                    all_reviews.extend(data)
                    page += 1
                    time.sleep(random.uniform(1, 3))
                else:
                    break
            else:
                print(f"Fetching {critique_url} is not allowed by robots.txt")
                break

    save_data(all_reviews, 'reviews_with_ratings_2.json')
    print(f"nombre de commentaires positifs: {counts[1]}, nombre de commentaires neutres: {counts[0]}, nombre de commentaires négatifs: {counts[-1]}")

if __name__ == "__main__":
    main()

Error fetching robots.txt from https://www.babelio.com/robots.txt: HTTPSConnectionPool(host='www.babelio.com', port=443): Max retries exceeded with url: /robots.txt (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fabd87436d0>: Failed to establish a new connection: [Errno 60] Operation timed out'))
Fetching https://www.babelio.com/livres/Rousselet-La-belle-histoire-des-maths/121795/critiques?a=a&pageN=1 is not allowed by robots.txt
Error fetching robots.txt from https://www.babelio.com/robots.txt: HTTPSConnectionPool(host='www.babelio.com', port=443): Max retries exceeded with url: /robots.txt (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fabca007640>: Failed to establish a new connection: [Errno 60] Operation timed out'))
Fetching https://www.babelio.com/livres/Saint-Exupery-Le-Petit-Prince/36712/critiques?a=a&pageN=1 is not allowed by robots.txt
Error fetching robots.txt from https://www.babelio.com/robots.txt: HTTPSCon